In [1]:
# Imports

import pandas as pd
import numpy as np

import json
from pathlib import Path

In [2]:
# Caminhos dos arquivos

sep = "\t"
encoding = "utf-8"

E3_056_Csv = "E3-056_Chuva_Diaria_Serie_20240918_122745.csv"
E3_062_Csv = "E3-062_Chuva_Diaria_Serie_20240918_122827.csv"
E3_063_Csv = "E3-063_Chuva_Diaria_Serie_20240918_122846.csv"
E3_064_Csv = "E3-064_Chuva_Diaria_Serie_20240918_122858.csv"
E3_065_Csv = "E3-065_Chuva_Diaria_Serie_20240918_122918.csv"
E3_066_Csv = "E3-066_Chuva_Diaria_Serie_20240918_122928.csv"
E3_228_Csv = "E3-228_Chuva_Diaria_Serie_20240918_122948.csv"

csv_archive = E3_056_Csv

path_to_archive = f"C:/Users/Kelvin/Desktop/GA/governo_aberto/dados/{csv_archive}"

do_print = True
if do_print:
    print(f"Path to csv input is:  {path_to_archive}")

Path to csv input is:  C:/Users/Kelvin/Desktop/GA/governo_aberto/dados/E3-056_Chuva_Diaria_Serie_20240918_122745.csv


In [3]:
# Funcao Auxiliar

def extrair_identificador(csv_archive):
    return csv_archive.split('_')[0]

In [4]:
# Funcoes Media (Periodo Todo - Decada - Tres Decadas)

def calcular_media_periodo_completo(df, csv_archive):

    if 'Chuva total' not in df.columns:
        print("Coluna 'Chuva total' não encontrada")
        return None
    
    chuva_total = df['Chuva total'].dropna()
    
    if len(chuva_total) == 0:
        print("Nenhum dado válido encontrado")
        return None
    
    estatisticas = {
        "media_mensal": round(float(chuva_total.mean()), 2),
        "periodo": f"{df.iloc[0,0]} a {df.iloc[-1,0]}",
        "total_meses": int(len(df)),
        "meses_validos": int(len(chuva_total)),
        "meses_faltantes": int(len(df) - len(chuva_total)),
        "maximo": round(float(chuva_total.max()), 2),
        "minimo": round(float(chuva_total.min()), 2)
    }
    
    # Salvar em JSON
    json_filename = f"C:/Users/Kelvin/Desktop/GA/governo_aberto/resultados/Periodo_Todo/{csv_archive}_media_mensal_periodo_completo.json"
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(estatisticas, f, indent=2, ensure_ascii=False)
    
    print(f"Média do período completo salva em: {json_filename}")
    return estatisticas

def calcular_media_por_decada(df, csv_archive):

    if 'Chuva total' not in df.columns:
        print("Coluna 'Chuva total' não encontrada")
        return None
    
    # Extrair ano da coluna Mês/Ano
    def extrair_ano(data_str):
        try:
            return int(data_str.split('/')[1])
        except:
            return None
    
    df_copy = df.copy()
    df_copy['Ano'] = df_copy['Mês/Ano'].apply(extrair_ano)
    df_copy = df_copy.dropna(subset=['Ano'])
    
    # Criar décadas (1941-1950, 1951-1960, etc)
    estatisticas_decadas = {}
    
    # Encontrar range de anos
    anos = df_copy['Ano'].unique()
    ano_min = int(min(anos))
    ano_max = int(max(anos))
    
    # Ajustar para começar em 1941
    decada_inicio = 1941
    if ano_min > 1941:
        decada_inicio = ((ano_min - 1) // 10) * 10 + 1
    
    # Calcular para cada década
    decada_atual = decada_inicio
    while decada_atual <= ano_max:
        decada_fim = decada_atual + 9
        mask = (df_copy['Ano'] >= decada_atual) & (df_copy['Ano'] <= decada_fim)
        df_decada = df_copy[mask]
        
        chuva_decada = df_decada['Chuva total'].dropna()
        
        if len(chuva_decada) > 0:
            estatisticas_decadas[f"{decada_atual}-{decada_fim}"] = {
                "media_mensal": round(float(chuva_decada.mean()), 2),
                "total_meses": int(len(df_decada)),
                "meses_validos": int(len(chuva_decada)),
                "maximo": round(float(chuva_decada.max()), 2),
                "minimo": round(float(chuva_decada.min()), 2)
            }
        
        decada_atual += 10
    
    # Salvar em JSON
    json_filename = f"C:/Users/Kelvin/Desktop/GA/governo_aberto/resultados/Decadas/{csv_archive}_media_mensal_decada.json"
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(estatisticas_decadas, f, indent=2, ensure_ascii=False)
    
    print(f"Médias por década salvas em: {json_filename}")
    print(f"Décadas calculadas: {list(estatisticas_decadas.keys())}")
    
    return estatisticas_decadas

def calcular_media_por_tres_decadas(df, csv_archive):

    if 'Chuva total' not in df.columns:
        print("Coluna 'Chuva total' não encontrada")
        return None
    
    # Extrair ano da coluna Mês/Ano
    def extrair_ano(data_str):
        try:
            return int(data_str.split('/')[1])
        except:
            return None
    
    df_copy = df.copy()
    df_copy['Ano'] = df_copy['Mês/Ano'].apply(extrair_ano)
    df_copy = df_copy.dropna(subset=['Ano'])
    
    # Criar períodos de três décadas
    estatisticas_tres_decadas = {}
    
    # Encontrar range de anos
    anos = df_copy['Ano'].unique()
    ano_min = int(min(anos))
    ano_max = int(max(anos))
    
    # Ajustar para começar em 1941
    periodo_inicio = 1941
    if ano_min > 1941:
        periodo_inicio = ((ano_min - 1) // 10) * 10 + 1
    
    # Calcular para cada período de três décadas
    periodo_atual = periodo_inicio
    while periodo_atual + 29 <= ano_max:  # 3 décadas = 30 anos - 1
        periodo_fim = periodo_atual + 29
        mask = (df_copy['Ano'] >= periodo_atual) & (df_copy['Ano'] <= periodo_fim)
        df_periodo = df_copy[mask]
        
        chuva_periodo = df_periodo['Chuva total'].dropna()
        
        if len(chuva_periodo) > 0:
            estatisticas_tres_decadas[f"{periodo_atual}-{periodo_fim}"] = {
                "total_meses": int(len(df_periodo)),
                "meses_validos": int(len(chuva_periodo)),
                "maximo": round(float(chuva_periodo.max()), 2),
                "minimo": round(float(chuva_periodo.min()), 2),
                "anos_cobertura": f"{periodo_atual}-{periodo_fim} ({periodo_fim - periodo_atual + 1} anos)"
            }
        
        periodo_atual += 10  # Janela deslizante de 10 anos
    
    # Salvar em JSON
    json_filename = f"C:/Users/Kelvin/Desktop/GA/governo_aberto/resultados/Tres_Decadas/{csv_archive}_media_mensal_tres_decadas.json"
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(estatisticas_tres_decadas, f, indent=2, ensure_ascii=False)
    
    print(f"Médias por três décadas salvas em: {json_filename}")
    print(f"Períodos calculados: {list(estatisticas_tres_decadas.keys())}")
    
    return estatisticas_tres_decadas

In [5]:
# Funcao Principal

def calcular_todas_medias(df, csv_archive):
    
    resultados = {}
    
    # 1. Média do período completo
    print("\n1 - Calculando média do período completo...")
    resultado_periodo = calcular_media_periodo_completo(df, csv_archive)
    resultados['periodo_completo'] = resultado_periodo
    
    # 2. Média por década
    print("\n2 - Calculando média por década...")
    resultado_decada = calcular_media_por_decada(df, csv_archive)
    resultados['por_decada'] = resultado_decada
    
    # 3. Média por três décadas
    print("\n3️ - Calculando média por três décadas...")
    resultado_tres_decadas = calcular_media_por_tres_decadas(df, csv_archive)
    resultados['por_tres_decadas'] = resultado_tres_decadas
    
    print("\nCÁLCULOS CONCLUÍDOS!")
    print("=" * 30)
    
    # Resumo dos resultados
    if resultado_periodo:
        print(f"Média período completo: {resultado_periodo['media_mensal']} mm")
    
    if resultado_decada:
        print(f"Décadas calculadas: {len(resultado_decada)}")
    
    if resultado_tres_decadas:
        print(f"Períodos de 3 décadas: {len(resultado_tres_decadas)}")
    
    return resultados

In [6]:
# 1 - Transformar o arquivo em um DataFrame mais legível

# Primeiro, carregar os dados como texto puro
df_raw = pd.read_csv(path_to_archive, 
                    sep=sep,
                    encoding=encoding,
                    skiprows=9,
                    header=0,
                    dtype=str)  # Manter tudo como string

print(f"Shape original: {df_raw.shape}")

# Extrair o cabeçalho da primeira linha
header_line = df_raw.iloc[0, 0]
print(f"Linha do cabeçalho: {header_line}")

# Dividir o cabeçalho por ponto e vírgula para obter os nomes das colunas
colunas = [col.strip() for col in header_line.split(';')]
print(f"Colunas identificadas: {colunas}")
print(f"Número de colunas: {len(colunas)}")

# Criar um novo DataFrame vazio com as colunas corretas
df_novo = pd.DataFrame(columns=colunas)

# Processar cada linha de dados
for i in range(1, len(df_raw)):
    linha = df_raw.iloc[i, 0]
    
    if pd.notna(linha): 
        # Dividir a linha por ponto e vírgula
        valores = [val.strip() for val in linha.split(';')]
        
        # Garantir que temos o número correto de colunas
        if len(valores) == len(colunas):
            df_novo.loc[len(df_novo)] = valores
        else:
            print(f"Linha {i} tem {len(valores)} valores, esperado {len(colunas)}")
            # Preencher com NaN se faltarem colunas
            valores.extend([np.nan] * (len(colunas) - len(valores)))
            df_novo.loc[len(df_novo)] = valores

print(f"Novo DataFrame criado - Shape: {df_novo.shape}")
print(f"Período: {df_novo.iloc[0,0]} a {df_novo.iloc[-1,0]}")

Shape original: (877, 1)
Linha do cabeçalho: Mês/Ano;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;Chuva máxima;Chuva total
Colunas identificadas: ['Mês/Ano', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', 'Chuva máxima', 'Chuva total']
Número de colunas: 34
Novo DataFrame criado - Shape: (876, 34)
Período: 10/1938 a 06/2023


In [7]:
# 2 - Inserção de todos os dados

# Limpeza dos dados
print("\nRealizando limpeza dos dados...")

# Converter colunas numéricas (dias 1-31 e chuvas)
for col in df_novo.columns[1:]:  # Pular a coluna 'Mês/Ano'
    # Substituir '---' por NaN
    df_novo[col] = df_novo[col].replace('---', np.nan)
    
    # Converter vírgula para ponto (padrão float)
    df_novo[col] = df_novo[col].str.replace(',', '.', regex=False)
    
    # Converter para numérico
    df_novo[col] = pd.to_numeric(df_novo[col], errors='coerce')

print("Dados limpos e convertidos")

# Verificar resultado
print(f"\nPrimeiras 3 linhas do novo DataFrame:")
print(df_novo.head(3))

print(f"\nÚltimas 3 linhas:")
print(df_novo.tail(3))

print(f"\nInfo do DataFrame:")
print(df_novo.info())


Realizando limpeza dos dados...
Dados limpos e convertidos

Primeiras 3 linhas do novo DataFrame:
   Mês/Ano    1    2    3    4    5    6    7     8     9  ...    24   25  \
0  10/1938  0.0  0.0  3.4  0.2  0.4  2.5  0.0   0.0   0.0  ...   0.0  0.0   
1  11/1938  1.4  0.0  0.0  0.0  0.0  0.0  0.0   2.1   0.0  ...   1.0  0.6   
2  02/1939  0.0  0.0  0.0  0.0  0.0  0.0  0.0  49.0  51.0  ...  53.0  0.0   

    26   27   28     29   30   31  Chuva máxima  Chuva total  
0  0.0  0.0  0.0  102.0  0.0  0.0         102.0        117.6  
1  0.0  2.0  0.8    0.0  0.4  NaN           6.3         26.7  
2  0.0  0.0  0.0    NaN  NaN  NaN          56.0        323.0  

[3 rows x 34 columns]

Últimas 3 linhas:
     Mês/Ano     1    2    3    4    5    6    7    8    9  ...  24  25  26  \
873  04/2023   0.0  0.0  2.6  NaN  NaN  NaN  NaN  NaN  NaN  ... NaN NaN NaN   
874  05/2023   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ... NaN NaN NaN   
875  06/2023  33.4  0.0  0.0  0.0  0.7  0.0  0.0  0.0  0.0  .

In [8]:
# 2.1 Removver a precipitacao do dia 18.02.2021 (Nao faz diferenca kk)

linha_fevereiro_2021 = df_novo[df_novo['Mês/Ano'] == '02/2021']
    
if len(linha_fevereiro_2021) > 0:
    idx = linha_fevereiro_2021.index[0]
    
    # Verificar se a coluna '18' existe
    if '18' in df_novo.columns:
            # Alterar o valor para NaN
            valor_antigo = df_novo.loc[idx, '18']
            df_novo.loc[idx, '18'] = np.nan
            
            print(f"ALTERAÇÃO REALIZADA COM SUCESSO:")
            print(f"   - Linha: {idx} (Mês/Ano: 02/2021)")
            print(f"   - Coluna: '18' (dia 18)")
            print(f"   - Valor anterior: {valor_antigo}")
            print(f"   - Valor novo: {df_novo.loc[idx, '18']}")
            
            # Verificação adicional
            print(f"\nVerificação:")
            print(f"Valor na linha {idx}, coluna '18': {df_novo.loc[idx, '18']}")
    else:
        print("Coluna '18' não encontrada")
        print("Colunas disponíveis:")
        for col in df_novo.columns:
            print(f"  - {col}")
       

ALTERAÇÃO REALIZADA COM SUCESSO:
   - Linha: 848 (Mês/Ano: 02/2021)
   - Coluna: '18' (dia 18)
   - Valor anterior: 706.0
   - Valor novo: nan

Verificação:
Valor na linha 848, coluna '18': nan


In [9]:
# 3 - Execucao

if 'Chuva total' in df_novo.columns:
    identificador = extrair_identificador(csv_archive)

    # Calcular todas as médias de uma vez
    # resultados = calcular_todas_medias(df_novo, identificador)

    # Ou calcular individualmente
    resultado1 = calcular_media_periodo_completo(df_novo, identificador)
    resultado2 = calcular_media_por_decada(df_novo, identificador)
    resultado3 = calcular_media_por_tres_decadas(df_novo, identificador)

Média do período completo salva em: C:/Users/Kelvin/Desktop/GA/governo_aberto/resultados/Periodo_Todo/E3-056_media_mensal_periodo_completo.json
Médias por década salvas em: C:/Users/Kelvin/Desktop/GA/governo_aberto/resultados/Decadas/E3-056_media_mensal_decada.json
Décadas calculadas: ['1941-1950', '1951-1960', '1961-1970', '1971-1980', '1981-1990', '1991-2000', '2001-2010', '2011-2020']
Médias por três décadas salvas em: C:/Users/Kelvin/Desktop/GA/governo_aberto/resultados/Tres_Decadas/E3-056_media_mensal_tres_decadas.json
Períodos calculados: ['1941-1970', '1951-1980', '1961-1990', '1971-2000', '1981-2010', '1991-2020']
